

# Побудувати просту систему рекомендацій аніме
Треба побудувати рекомендації аніме по схемі "Разом з цим дивились"

### Датасет
Для початку качаємо датасет за <a scr="https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database">посиланням</a> (Потрбна реестрація).
і розпаковуємо в папку з проектом

В нас є два файли `Anime.csv` і `Rating.csv`

`Anime.csv` має формат:
* **anime_id** - унікальний ідентифікатор, який ідентифікує аніме.
* **name** - повна назва аніме.
* **genre** - розділений комами список жанрів для цього аніме.
* **type** - фільм, ТБ, OVA тощо.
* **episodes** - скільки епізодів у цьому шоу. (1 якщо фільм).
* **rating** - середня оцінка з 10 для цього аніме.
* **members** - кількість членів спільноти, які входять до «групи» цього аніме.

`Rating.csv` має формат:
* **user_id** - не ідентифікований випадково згенерований ідентифікатор користувача.
* **anime_id** - аніме, яке оцінив цей користувач.
* **rating** - оцінка з 10, призначена цим користувачем (-1, якщо користувач дивився, але не поставив оцінку).

### Аналіз датасету:
* Порахувати кількість унікальних user_id, anime_id та звязків
* Вирахувати топ 100 самих популярних аніме
* Вирахувати середній рейтінг по аніме та середній рейтінг для кожного юзера (не враховувати -1)

### Підготовка датасету:
* Читаємо файл rating.csv, розбиваємо по "\n" (переносу на новий рядок) і формуємо список зі строчок. Слід зауважати що перший рядочок з назвою колонок треба пропустити. Далі для кожного елементу списку розбиваемо сточку по розділювальному знаку "," і отримуємо список зі списків, де перший елемент це id юзера, другий - id аніме, 3 - оцінка
* Пройтись по отрманому списку і сформувати словник, де ключом буде user_id, а значенням список із anime_id, які він дивився
```
{
    "user_id_1": [anime_id_1, anime_id_2, ... ],
    "user_id_2": [anime_id_2, anime_id_4, ... ],
    ...
}
```
 * Розбити датасет на 2 підвіборки train та test
     * за допомогою slice (просте) (для початку робимо так)
     * за допомогою random (середне)
     * за допомогою sklern по юзерам (складніше)

### Побудова простої рекомендації (по train):

* Після цього переформотувати цей словник в новий словник, де ключом буде аніме_id_n, а значенням словник з ключом anime_id_m а значенням кількість юзерів які дивились разом (будемо називати це rank) з аніме_id_n і аніме_id_m
```
{
    "anime_id_1": {
        "anime_id_2": 2,
        "anime_id_3": 3,
        ...
    },
    "anime_id_2": {
        "anime_id_2": 1,
        "anime_id_1": 3,
        ...
    },
    ...
}
```
* Зберігти в форматі json отримані рекомендації


### Predict
Як зробити рекомендацію anime по тому яке ви дивились:
* Підзавантажити отримані рекомендації з json файлу
* Читаємо файл anime.csv, розбиваємо по "\n" (переносу на новий рядок) і формуємо список зі строчок. Слід зауважати що перший рядочок з назвою колонок треба пропустити. Далі для кожного елементу списку розбиваемо сточку по розділювальному знаку "," і отримуємо список зі списків, де перший елемент це id anime, другий - назва аніме і так далі
* Пройтись по отрманому списку і сформувати два словники, Перший: ключ - anime_name, значенням - anime_id. Другий: ключ - anime_id, значенням - anime_name. 
* Обрати зі списку аніме яке вам подобаеться. Напиклад: "One Piece". По назві аніме з словника отримати anime_id
* По anime_id отримати зі словника рекомендацій схожі anime_id(s). Відсортувати їх в порядку спадання.
* По словнику(ключ - anime_id, значенням - anime_name) отримати і вивести назви рекомендуемих аніме

### Predict batch
Як зробити рекомендацію anime по декільком аніме які ви(користувач) дивились
* Для кожного anime_id отримати зі словника рекомендацій схожі anime_id(s)
* Всі отримані рекомендації змерджити в один словник просумувавши всі rank
* Відсортувати отрмані айтеми словника по спаданню 
* Вивести максимум топ 100 рекомендуемих аніме


### Evaluate
Ця секція про тестування (в контексті ml) нашого рішення на датасеті test
Тут ми хочемо перевірити наскільки добре працює наше рішення

Якщо ми тестуемо це по 1 аніме

Ми можемо взяти кожного юзера і перше anime_id в списку яке він дивився вважати що ми дійсно знаемо на данний момент часу, що він його дивився, а всі інші вважати, що він поживиться пісця цього моменту часу
Тоді ми можемо порівняти всі інші anime які він дивився з рекомендаціями по першому аніме (Топ 10)

Наприклад:
Ми знаемо що Юзер1 дивився anime1,anime2,anime3,anime4,anime5.
Беремо рекомендації по anime1 - нехай це буде anime2,anime6,anime4,anime7.
Тоді рахуемо кількість аніме яку б цей юзер точно подивився (anime2, anime4) 2.
І кількість яку ми не знаемо чи подиввся би (anime6, anime7) 2.
Тоді точніть по цьому юзеру наших рекомендацій буде 2/(2+2) = 0.5.
Так треба порахувати для кожного юзера і взяти від цього середне

In [ ]:
# https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/Content%20Based%20Recommendation%20Engines%20using%20Python.ipynb
